# SK Memory - RAG Patter Primer

Reference:
- https://github.com/Azure-Samples/azure-search-openai-demo
- https://github.com/azure-samples/semantic-kernel-rag-chat

## Load the required .NET packages and supporting classes

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.18.230725.3-preview"
#r "nuget: Microsoft.Extensions.Logging.Console"
#r "nuget: dotenv.net"

using System.ComponentModel;
using System.Net.Http;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.SkillDefinition;
using Microsoft.SemanticKernel.Skills.Core;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Utils/ConsoleLogger.cs

const string MemoryCollectionName = "aboutMe";

Installed Packages dotenv.net, 3.1.2 Microsoft.Extensions.Logging.Console, 7.0.0 Microsoft.SemanticKernel, 0.18.230725.3-preview

## Load the OpenAI variables from environment variables or an .env file

In [2]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var adaDeploymentName = "ada";
Console.WriteLine($"Using deployment: {deploymentName} at: {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at: https://alemorfr.openai.azure.com/ with key 32c87...


## Get a Kernel with an embedding service

In [3]:
var kernel = Kernel.Builder
            .WithLogger(ConsoleLogger.Log)
            .WithAzureChatCompletionService(deploymentName, endpoint, apiKey)
            .WithAzureTextEmbeddingGenerationService(adaDeploymentName, endpoint, apiKey)
            .WithMemoryStorage(new VolatileMemoryStore())
            .Build();

## Save memories

In [4]:
// ========= Store memories using the kernel =========

await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andre");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I work as a tourist operator");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I've been living in Seattle since 2005");
await kernel.Memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");

## Record memory using a Skill

In [5]:
// ========= Store memories using semantic function =========

// Add Memory as a skill for other functions
var memorySkill = new TextMemorySkill(kernel.Memory);
kernel.ImportSkill(memorySkill);

// Build a semantic function that saves info to memory
const string SaveFunctionDefinition = "{{save $info}}";
var memorySaver = kernel.CreateSemanticFunction(SaveFunctionDefinition);

var context = kernel.CreateNewContext();
context[TextMemorySkill.CollectionParam] = MemoryCollectionName;
context[TextMemorySkill.KeyParam] = "info5";
context["info"] = "My family is from New York";
await memorySaver.InvokeAsync(context);

## Retrive a memory

In [6]:
// ========= Test memory remember =========
Console.WriteLine("========= Example: Recalling a Memory =========");

var answer = await memorySkill.RetrieveAsync(MemoryCollectionName, "info5", logger: context.Log);
Console.WriteLine("Memory associated with 'info1': {0}", answer);

========= Example: Recalling a Memory =========
Memory associated with 'info1': My family is from New York


## Test a memory recall

In [7]:
Console.WriteLine("========= Example: Recalling an Idea =========");

answer = await memorySkill.RecallAsync("where did I grow up?", MemoryCollectionName, relevance: null, limit: 2, logger: context.Log);
Console.WriteLine("Ask: where did I grow up?");
Console.WriteLine("Answer:\n{0}", answer);

answer = await memorySkill.RecallAsync("where do I live?", MemoryCollectionName, relevance: null, limit: 2, logger: context.Log);
Console.WriteLine("Ask: where do I live?");
Console.WriteLine("Answer:\n{0}", answer);

========= Example: Recalling an Idea =========
Ask: where did I grow up?
Answer:
["My family is from New York","I\u0027ve been living in Seattle since 2005"]
Ask: where do I live?
Answer:
["I\u0027ve been living in Seattle since 2005","My family is from New York"]


## Use an SK function to recall a memory

In [8]:
Console.WriteLine("========= Example: Using Recall in a Semantic Function =========");

// Build a semantic function that uses memory to find facts
const string RecallFunctionDefinition = @"
Consider only the facts below when answering questions.

About me: {{recall 'where did I grow up?'}}
About me: {{recall 'where do I live?'}}

Question: {{$input}}

Answer:
";

var aboutMeOracle = kernel.CreateSemanticFunction(RecallFunctionDefinition, maxTokens: 100);

context = kernel.CreateNewContext();
context[TextMemorySkill.CollectionParam] = MemoryCollectionName;
context[TextMemorySkill.RelevanceParam] = "0.8";
var result = await aboutMeOracle.InvokeAsync("Do I live in the same town where I grew up?", context);

Console.WriteLine("Do I live in the same town where I grew up?\n");
Console.WriteLine(result);

========= Example: Using Recall in a Semantic Function =========
Do I live in the same town where I grew up?

The given facts do not provide information about where you grew up, so it is not possible to determine whether you currently live in the same town.


## Remove a memory

In [9]:
Console.WriteLine("========= Example: Forgetting a Memory =========");

context["fact1"] = "What is my name?";
context["fact2"] = "What do I do for a living?";
context[TextMemorySkill.RelevanceParam] = ".75";

result = await aboutMeOracle.InvokeAsync("Tell me a bit about myself", context);

Console.WriteLine("Tell me a bit about myself\n");
Console.WriteLine(result);

context[TextMemorySkill.KeyParam] = "info1";
await memorySkill.RemoveAsync(MemoryCollectionName, "info1", logger: context.Log);

result = await aboutMeOracle.InvokeAsync("Tell me a bit about myself", context);

Console.WriteLine("Tell me a bit about myself\n");
Console.WriteLine(result);

========= Example: Forgetting a Memory =========
Tell me a bit about myself

Based on the given facts, you are originally from New York and have been living in Seattle since 2005.
Tell me a bit about myself

Based on the given facts, you are originally from New York and have been living in Seattle since 2005.
